# DS 310 HW 2

Author: Jess Strait

Consider the homeless youth dataset on canvas. It contains lots of features and information on different kinds of drug usage behaviors of homeless youth. In this problem, we consider marijuana usage. Just categorize all the datapoints in the dataset into marijuana users and non-users. Once this is done, run logistic regression using scikitlearn and Jupyter notebook with any features of your choice. You should be able to achieve an accuracy of greater than 0.6.

In [145]:
# Import useful packages
%reset
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [146]:
# Import, format, and explore dataset
youth = pd.read_excel("originaldata.xlsx", sep = ';')
print(youth.keys())
print(youth.info())

x = pd.DataFrame(youth)
print(x.head())

Index(['pid', 'screen1_sleep', 'screen1_sleep_18_text', 'screen_voucher',
       'screen2_long', 'screen3_age', 'realm_score', 'consent',
       'consent_agree', 'pid_20',
       ...
       'new_helpseek_scale_1', 'new_helpseek_scale_2', 'new_helpseek_scale_3',
       'new_helpseek_scale_4', 'new_helpseek_scale_5', 'new_helpseek_scale_6',
       'new_helpseek_scale_7', 'new_helpseek_scale_8', 'sum_helpseek_scale',
       'erorhosp_ever'],
      dtype='object', length=1210)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6920 entries, 0 to 6919
Columns: 1210 entries, pid to erorhosp_ever
dtypes: float64(1112), int64(70), object(28)
memory usage: 63.9+ MB
None
    pid  screen1_sleep screen1_sleep_18_text  screen_voucher  screen2_long  \
0  1004             15                     .             NaN           NaN   
1  1004             15                     .             NaN           NaN   
2  1004             15                     .             NaN           NaN   
3  1004            

In [147]:
# Convert target to boolean variable where 1 = marijuana user
x.druguse_30_6 = (x.druguse_30_6 != 1).astype(int)
x.druguse_30_6.value_counts()

0    4840
1    2080
Name: druguse_30_6, dtype: int64

In [148]:
# Double check that conversion worked correctly by verifying with original dataset
print(x.druguse_30_6.head())

0    0
1    0
2    0
3    0
4    0
Name: druguse_30_6, dtype: int32


In [149]:
# Identify features for feature selection by analyzing correlations between features and target variables
correlations = x.corr()
print(correlations['druguse_30_6'])

pid                     0.033234
screen1_sleep           0.054285
screen_voucher         -0.134840
screen2_long           -0.008499
screen3_age             0.022327
                          ...   
new_helpseek_scale_6    0.090154
new_helpseek_scale_7    0.053058
new_helpseek_scale_8    0.096776
sum_helpseek_scale      0.092213
erorhosp_ever           0.043746
Name: druguse_30_6, Length: 1182, dtype: float64


In [150]:
# Perform feature selection based on correlation results
corr = correlations['druguse_30_6'].sort_values()
top_corr = corr[:50]
labels = top_corr.axes
print(labels)
# Create subset of data frame using selected features
subx = x[['doctor_hcv', 'giscale_2', 'alt_dpvsynmj_di', 'pdmfranyfrandrev',
       'podcardfrdoc', 'alt_disclosesuithtontime', 'pdmfreefrfdandrev',
       'alt_tlkhepc', 'si', 'perceivhiv_2', 'q32_1', 'freefrirel',
       'gang_prov_money', 'q8_1', 'vict_nosexeam_any', 'gang_male', 'q39_4',
       'pdmbuyfrdealer', 'gang_prov_protect', 'screen_voucher', 'purdealer',
       'gang_prov_socspt', 'q39_2', 'alt_tlksafesex', 'q31_1',
       'gang_prov_safety', 'accsmphone', 'alt_dpvpdm_di', 'q44_1', 'q34_1',
       'discrim_reasons_12_di', 'q35_1', 'q39_1', 'q39_3', 'ex_debute',
       'gang_prov_shelter', 'jugg_age_1', 'marj_access_5',
       'a12monthhomeless_8', 'perceivhiv_1', 'gang_prov_clothes',
       'techaccess_1', 'new_perc_stress3', 'ever_sextradetraffic',
       'techaccess_3', 'q38_1', 'alt_disclosesuitht', 'sedatives',
       'alt_dpvconcur', 'gang_prov_food']]
print(subx.head())

[Index(['doctor_hcv', 'giscale_2', 'alt_dpvsynmj_di', 'pdmfranyfrandrev',
       'podcardfrdoc', 'alt_disclosesuithtontime', 'pdmfreefrfdandrev',
       'alt_tlkhepc', 'si', 'perceivhiv_2', 'q32_1', 'freefrirel',
       'gang_prov_money', 'q8_1', 'vict_nosexeam_any', 'gang_male', 'q39_4',
       'pdmbuyfrdealer', 'gang_prov_protect', 'screen_voucher', 'purdealer',
       'gang_prov_socspt', 'q39_2', 'alt_tlksafesex', 'q31_1',
       'gang_prov_safety', 'accsmphone', 'alt_dpvpdm_di', 'q44_1', 'q34_1',
       'discrim_reasons_12_di', 'q35_1', 'q39_1', 'q39_3', 'ex_debute',
       'gang_prov_shelter', 'jugg_age_1', 'marj_access_5',
       'a12monthhomeless_8', 'perceivhiv_1', 'gang_prov_clothes',
       'techaccess_1', 'new_perc_stress3', 'ever_sextradetraffic',
       'techaccess_3', 'q38_1', 'alt_disclosesuitht', 'sedatives',
       'alt_dpvconcur', 'gang_prov_food'],
      dtype='object')]
   doctor_hcv  giscale_2  alt_dpvsynmj_di  pdmfranyfrandrev  podcardfrdoc  \
0         1.0       

In [151]:
# Fill na columns with mode for that feature
for column in subx:
    subx[column].fillna(subx[column].mode()[0], inplace=True)
print(subx.head())

# Check for remaining missing entries
print(subx.isnull().sum())

   doctor_hcv  giscale_2  alt_dpvsynmj_di  pdmfranyfrandrev  podcardfrdoc  \
0         1.0        3.0              1.0               1.0           1.0   
1         1.0        3.0              1.0               1.0           1.0   
2         1.0        3.0              1.0               1.0           1.0   
3         1.0        3.0              1.0               1.0           1.0   
4         1.0        3.0              1.0               1.0           1.0   

   alt_disclosesuithtontime  pdmfreefrfdandrev  alt_tlkhepc  si  perceivhiv_2  \
0                       0.0                0.0          0.0   0           5.0   
1                       0.0                0.0          0.0   0           5.0   
2                       0.0                0.0          0.0   0           5.0   
3                       0.0                0.0          0.0   0           5.0   
4                       0.0                0.0          1.0   0           5.0   

   ...  gang_prov_clothes  techaccess_1  new_perc_

C:\Users\jessl\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [152]:
# Prep conditioning variable
y = x['druguse_30_6']
print(y.head())

0    0
1    0
2    0
3    0
4    0
Name: druguse_30_6, dtype: int32


In [153]:
# Do minmax normalization as taught in class with preprocessing tools
x = subx.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
# x_scaled only needs the columns from subx data frame
x_scaled = pd.DataFrame(x_scaled, columns=['doctor_hcv', 'giscale_2', 'alt_dpvsynmj_di', 'pdmfranyfrandrev',
       'podcardfrdoc', 'alt_disclosesuithtontime', 'pdmfreefrfdandrev',
       'alt_tlkhepc', 'si', 'perceivhiv_2', 'q32_1', 'freefrirel',
       'gang_prov_money', 'q8_1', 'vict_nosexeam_any', 'gang_male', 'q39_4',
       'pdmbuyfrdealer', 'gang_prov_protect', 'screen_voucher', 'purdealer',
       'gang_prov_socspt', 'q39_2', 'alt_tlksafesex', 'q31_1',
       'gang_prov_safety', 'accsmphone', 'alt_dpvpdm_di', 'q44_1', 'q34_1',
       'discrim_reasons_12_di', 'q35_1', 'q39_1', 'q39_3', 'ex_debute',
       'gang_prov_shelter', 'jugg_age_1', 'marj_access_5',
       'a12monthhomeless_8', 'perceivhiv_1', 'gang_prov_clothes',
       'techaccess_1', 'new_perc_stress3', 'ever_sextradetraffic',
       'techaccess_3', 'q38_1', 'alt_disclosesuitht', 'sedatives',
       'alt_dpvconcur', 'gang_prov_food'])

# Concatenate with previously established target variable
final_dataset = pd.concat([x_scaled, y], axis=1, sort=False)
print(final_dataset)

# Double check for missing entries in the dataset features
final_dataset.isnull().sum()

      doctor_hcv  giscale_2  alt_dpvsynmj_di  pdmfranyfrandrev  podcardfrdoc  \
0            0.0        0.5              1.0               1.0           1.0   
1            0.0        0.5              1.0               1.0           1.0   
2            0.0        0.5              1.0               1.0           1.0   
3            0.0        0.5              1.0               1.0           1.0   
4            0.0        0.5              1.0               1.0           1.0   
...          ...        ...              ...               ...           ...   
6915         0.0        1.0              0.0               1.0           0.0   
6916         0.0        1.0              0.0               1.0           0.0   
6917         0.0        1.0              0.0               1.0           0.0   
6918         0.0        1.0              1.0               1.0           0.0   
6919         0.0        1.0              0.0               1.0           0.0   

      alt_disclosesuithtontime  pdmfree

doctor_hcv                  0
giscale_2                   0
alt_dpvsynmj_di             0
pdmfranyfrandrev            0
podcardfrdoc                0
alt_disclosesuithtontime    0
pdmfreefrfdandrev           0
alt_tlkhepc                 0
si                          0
perceivhiv_2                0
q32_1                       0
freefrirel                  0
gang_prov_money             0
q8_1                        0
vict_nosexeam_any           0
gang_male                   0
q39_4                       0
pdmbuyfrdealer              0
gang_prov_protect           0
screen_voucher              0
purdealer                   0
gang_prov_socspt            0
q39_2                       0
alt_tlksafesex              0
q31_1                       0
gang_prov_safety            0
accsmphone                  0
alt_dpvpdm_di               0
q44_1                       0
q34_1                       0
discrim_reasons_12_di       0
q35_1                       0
q39_1                       0
q39_3     

In [154]:
# Prepare data for split

x = final_dataset.loc[:,final_dataset.columns!='druguse_30_6']
y = final_dataset['druguse_30_6']

# Split training and testing data
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

# Prep and fit model
model = LogisticRegression(random_state=0)
print(model)

model.fit(x_train,y_train)

# Report trained model information
print("Coefficients: "+str(model.coef_)+" Intercept: "+str(model.intercept_))

# Make predictions with testing data
predicted = model.predict(x_test)
# Calculate probabilities as shown in class
probs = model.predict_proba(x_test.to_numpy())

x_test['Predictions'] = predicted
x_test['Actual'] = y_test
x_test['Predicted_Prob'] = probs[:,1]

# Visually examine results
print(x_test.head(15))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Coefficients: [[-2.28929786 -0.31166009 -0.68398714 -1.4787408  -0.9192021  -0.92778354
   0.38519297  0.62905547 -0.57261811 -0.26946722  0.06340408  0.38519297
  -0.46636308  0.19039372 -1.79366564 -0.13304994 -0.57586304 -0.32579637
  -0.1647667   0.33005761 -0.32579637 -0.75202516 -1.075713    0.32713932
  -0.53699356  0.09611644 -0.41536448  0.23787303 -0.99363248  0.98480314
  -0.31189976 -0.07743904 -1.14888488 -1.0946024  -0.31352254 -0.22316418
  -0.66633224  2.39183608 -0.36349842 -0.0798684   0.29133596  0.13521785
  -0.44576981 -0.36569682 -0.10111668  0.73459934  1.08040532  0.21011399
  -1.5718308  -0.13516275]] Intercept: [7.95598555]
      doctor_hcv  g

C:\Users\jessl\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\jessl\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jessl\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying 

In [155]:
# Compute confusion matrix
print(confusion_matrix(y_test, predicted))

[[1125   76]
 [ 354  175]]


In [156]:
# Compute accuracy score
print(accuracy_score(y_test, predicted))

0.7514450867052023
